# Training Workflow

Here we show the process of processing a query request.It includes the following steps：
    1. Connect DO nodes and MPC nodes.
    2. Load model.
    3. Train model.
    4. return result.

### Preparation

We need start the websocket server workers.

### Step 1: get info from response

由于我们需要展示一个处理查询的流程，因此我们首先需要创建一个包含查询信息的结构体。
首先导入依赖包。

In [3]:
import datetime
import sys
import socket

import syft
import torch

sys.path.append('../../../')
from soterml.connection.proto import soterone_data_pb2, soterone_service_pb2
from soterml.learning.horizontal.torch_model.testnet_models import Net1
from soterml.connection.chain import SoterOneChain
from soterml.learning.testnet.utils import get_nodes_from_query_info
from soterml.learning.testnet.MNIST_MPC_websocket_parallel import train_model


def get_host_ip():
    """
    query native IP
    :return: ip
    """
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    finally:
        s.close()

    return ip


def get_query_info_and_nodes_info():
    cur_time = str(datetime.date.today())
    mock_response = soterone_service_pb2.GetQueryInfoResponse()
    query_info = mock_response.query_info
    query_info.uuid = 'uuid ' + cur_time
    query_info.content = ('content ' + cur_time).encode('utf8')

    query_cluster = mock_response.query_cluster
    query_cluster.query_uuid = 'query_cluster.query_uuid ' + cur_time

    # MPC nodes
    mpc_nodes = query_cluster.mpc_nodes.add()
    mpc_nodes.address = get_host_ip()
    mpc_nodes.port = "8779"

    mpc_nodes = query_cluster.mpc_nodes.add()
    mpc_nodes.address = get_host_ip()
    mpc_nodes.port = "8780"

    mpc_nodes = query_cluster.mpc_nodes.add()
    mpc_nodes.address = get_host_ip()
    mpc_nodes.port = "8781"

    # Compute nodes
    do_nodes = query_cluster.do_nodes.add()
    do_nodes.address = get_host_ip()
    do_nodes.port = "8777"

    do_nodes = query_cluster.do_nodes.add()
    do_nodes.address = get_host_ip()
    do_nodes.port = "8778"

    mock_response.query_info.data_set = 2

    return mock_response



In [4]:
from syft import VirtualWorker
query_info_and_nodes_info = get_query_info_and_nodes_info()
model = Net1()

hook = syft.TorchHook(torch)
compute_nodes = [VirtualWorker(id='worker1', hook=hook), VirtualWorker(id='worker2', hook=hook)]
mpc_nodes = [VirtualWorker(id='mpc1', hook=hook),
             VirtualWorker(id='mpc2', hook=hook),
             VirtualWorker(id='mpc3', hook=hook)]
ctx = {
            'config1': {'ip': '127.0.0.1', 'port': '80'},
            'config2': {'ip': '127.0.0.1', 'port': '8080'},
        }

chain = SoterOneChain('http://132.232.36.171:8645',
                      '980b26e2fa86b5a1ca0e3fd6eafcfd226fb868727522022289287ceb7f328768',
                      ctx)
query_uuid = "test_submit_query"
model_trained = train_model(chain,
                            query_uuid,
                            query_info_and_nodes_info,
                            compute_nodes,
                            model,
                            mpc_nodes)

chain.shutdown()

data_set =  2
12800
start time:  11:18:27
Epoch 1

Test set: Average loss: 0.4901, Accuracy: 8864/10000 (88.6400%)

Epoch 2

Test set: Average loss: 0.4251, Accuracy: 8970/10000 (89.7000%)

Epoch 3

Test set: Average loss: 0.3854, Accuracy: 9074/10000 (90.7400%)

Epoch 4

Test set: Average loss: 0.3748, Accuracy: 9100/10000 (91.0000%)

Epoch 5

Test set: Average loss: 0.3606, Accuracy: 9120/10000 (91.2000%)

end time:  11:41:09
